In [1]:
import os
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg19, VGG19_Weights
from torchvision import transforms
from PIL import Image
import pickle
import re




# Base directory containing the ten cities
base_dir = "./urban_data/tencities/GenAI_density"

# List of cities
cities = [
    'Singapore',
    'HongKong',
    'Munich',
    'Stockholm',
    'Chicago',
    'Orlando',
    'Kinshasa',
    'SaoPaulo',
    'Mexico',
    'Kigali'
]

# Define which file extensions to consider as images
image_extensions = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

# How many images to sample per city
sample_size = 1000

for city in cities:
    # Specifically look in {city}/satellite_images_z17
    city_satellite_dir = os.path.join(base_dir, city, "satellite_images_z17")
    if not os.path.isdir(city_satellite_dir):
        print(f"Warning: directory does not exist for {city}: {city_satellite_dir}")
        continue

    # Collect all image filepaths recursively from satellite_images_z17
    city_img_files = []
    for root, dirs, files in os.walk(city_satellite_dir):
        for filename in files:
            if filename.lower().endswith(image_extensions):
                full_path = os.path.join(root, filename)
                city_img_files.append(full_path)

    # Randomly shuffle and take up to 1000
    random.shuffle(city_img_files)
    selected_files = city_img_files[:sample_size]

    # Write these image paths to a JSON file within the city folder
    out_json = os.path.join(base_dir, city, f"{city}_img_paths_sf.json")
    with open(out_json, 'w') as outfile:
        json.dump(selected_files, outfile, indent=2)

    print(f"Saved {len(selected_files)} image paths for {city} -> {out_json}")




KeyboardInterrupt: 

In [2]:
import os
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg19, VGG19_Weights
from torchvision import transforms
from PIL import Image
import pickle
import re
# YOUR OWN GENERATED IMAGE DIRECTORY
showcase_dir = "/Users/wangzhuoyulucas/SMART /generatedImg/show-case-FID"

groupings_output = "/Users/wangzhuoyulucas/SMART /data_server/urban_data/tencities/GenAI_density/groupings_newFID.json"

# 1) Define multiple regex patterns
patterns = [
    # a) original pattern: something containing "_cv.jpg" and ending "_gt.png"
    re.compile(r'_cv\.jpg.*_gt\.png$', re.IGNORECASE),
    
    # b) pattern for e.g. "34_45_r0_d0_cv.jpggt-Chicago"
    #    or "61_52_r0_d1_cv.jpggt-Mexico.png"
    #    We look for "_cv.jpggt-" + (some city text) + optional extension
    #    NOTE: Adjust character class [^.]+ if city names can have spaces, etc.
    re.compile(r'_cv\.jpggt-[^.]+(\.png|\.jpg|\.jpeg|\.tiff|\.bmp)?$', re.IGNORECASE),
]

groupings_list = []

for fname in os.listdir(showcase_dir):
    # Skip directories
    if os.path.isdir(os.path.join(showcase_dir, fname)):
        continue
    
    # Check each pattern; if any matches, we collect it
    for pat in patterns:
        if pat.search(fname):
            groupings_list.append(fname)
            break  # stop checking other patterns once matched

# Save to JSON
with open(groupings_output, 'w') as f:
    json.dump(groupings_list, f, indent=2)

print(f"Found {len(groupings_list)} matching filenames.")
print(f"Groupings extracted and saved to: {groupings_output}")

Found 299 matching filenames.
Groupings extracted and saved to: /Users/wangzhuoyulucas/SMART /data_server/urban_data/tencities/GenAI_density/groupings_newFID.json


# Move files into new sampled folder (if needed)

In [ ]:
import os
import json
import shutil

# 1) List of city names and JSON filenames
cities = [
    "Singapore",
    "HongKong",
    "Munich",
    "Stockholm",
    "Chicago",
    "Orlando",
    "Kinshasa",
    "SaoPaulo",
    "Mexico",
    "Kigali"
]

# 2) Base path where the JSON files live, e.g.:
json_dir = "./urban_data/tencities/GenAI_density"

# 3) Output base directory for the final copied images
#    (adjust as needed)
destination_root = "./urban_data/tencities/GenAI_density/cities_sampled1"

# Ensure the final root directory exists
if not os.path.exists(destination_root):
    os.makedirs(destination_root)

# 4) Loop over the cities
for city in cities:
    # Construct the JSON path for each city
    json_filename = f"{city}/{city}_img_paths_sf.json"  
    json_path = os.path.join(json_dir, json_filename)

    if not os.path.isfile(json_path):
        print(f"[WARN] JSON file not found for {city}: {json_path}")
        continue

    # 5) Load the list of image paths
    with open(json_path, "r") as f:
        image_paths = json.load(f)  # a list of file paths (strings)

    # 6) Create the city-specific output folder
    city_output_folder = os.path.join(destination_root, city)
    os.makedirs(city_output_folder, exist_ok=True)

    # 7) Copy each image from its source to the city output folder
    print(f"[INFO] Copying {len(image_paths)} images for {city} into {city_output_folder} ...")
    for src_path in image_paths:
        if not os.path.isfile(src_path):
            print(f"  [WARN] Source image not found: {src_path}")
            continue

        # Construct a destination filename
        # If two files have the same basename, you might overwrite. 
        # Consider adding logic to handle duplicates (renaming, etc.).
        filename = os.path.basename(src_path)
        dst_path = os.path.join(city_output_folder, filename)

        # Copy file (including metadata)
        shutil.copy2(src_path, dst_path)

    print(f"[INFO] Done copying for {city}")

print("[INFO] All copying complete!")


In [ ]:
pip install pytorch-fid

# FID Calculation GPU (pls change your directory accordingly)

In [ ]:
import os
import subprocess
import pandas as pd
import torch  # Import torch to check GPU
from tqdm import tqdm

# **1) Define Paths**
base_real_dir = "./urban_data/tencities/GenAI_density/cities_sampled1"
base_generated_dir = "./urban_data/tencities/GenAI_density/cities_sampled/city_generated"

# **2) List of Cities**
cities = [
    "Singapore", "HongKong", "Munich", "Stockholm", "Chicago",
    "Orlando", "Kinshasa", "SaoPaulo", "Mexico", "Kigali"
]

# **3) Verify GPU is Available**
device = "cuda:9" if torch.cuda.is_available() else "cpu"
print(f"[INFO] Running on device: {device}")

# **4) Initialize Results Storage**
results = []

# **5) Loop through City Pairs & Compute FID**
for city in tqdm(cities, desc="Computing FID for city pairs"):
    real_folder = os.path.join(base_real_dir, city)
    generated_folder = os.path.join(base_generated_dir, f"{city}_generated")

    # **Check if folders exist**
    if not os.path.isdir(real_folder):
        print(f"[WARN] Real image folder missing: {real_folder}")
        continue
    if not os.path.isdir(generated_folder):
        print(f"[WARN] Generated image folder missing: {generated_folder}")
        continue

    # **Run pytorch_fid command**
    print(f"[INFO] Computing FID for {city} on device {device}...")
    try:
        cmd = [
            "python", "-m", "pytorch_fid",
            generated_folder, real_folder,
            "--device", device
        ]
        output = subprocess.run(cmd, capture_output=True, text=True)
        
        # Print raw output to check if GPU is detected
        print(output.stdout)

        # Extract FID score (last number from stdout)
        fid_score = None
        for line in output.stdout.split("\n"):
            if "FID:" in line:
                fid_score = float(line.split()[-1])  # Last value in the line

        results.append({"City": city, "FID": fid_score})
        print(f"[INFO] {city} -> FID: {fid_score}")

    except Exception as e:
        print(f"[ERROR] Failed to compute FID for {city}: {e}")
        results.append({"City": city, "FID": None})

# **6) Save FID Results to CSV**
df = pd.DataFrame(results)
output_csv = "./urban_data/tencities/GenAI_density/fid_results.csv"
df.to_csv(output_csv, index=False)

print(f"[INFO] FID scores saved to: {output_csv}")
